## UniRep

In [ ]:
from unirep.run_inference import BatchInference
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings

# 过滤掉所有DeprecationWarning
warnings.filterwarnings('ignore')

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
df_all = pd.read_json('../dataset/df_multi_tasks.json')
input_sequences = df_all['sequence'].drop_duplicates()
print(f'number of sequences : {len(input_sequences)}')
input_sequences.head()

In [2]:
protein_names, sequences = input_sequences.values, input_sequences.values

# convert format
seq_fasta_file_path = './unirep/tep.fasta'
fasta_file = open(seq_fasta_file_path, "w")
for name, seq in zip(protein_names, sequences):
    fasta_file.write(">" + name + "/n" +seq + "/n")
fasta_file.close()

# calculate
sess = tf.Session(config=config)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)  # 改变这个百分比即可
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
inf_obj = BatchInference(batch_size=2)
df_unirep_5400 = inf_obj.run_inference(filepath=seq_fasta_file_path)
df_unirep_5400.to_csv('./unirep/df_unirep.tsv', sep='/t')

# convert format
df_unirep_1900 = pd.read_csv('./unirep/df_unirep.tsv', sep='/t').drop(
    columns='Unnamed: 0').iloc[:, :1900]

unirep_values = df_unirep_1900.values.tolist()

unirep_col_value = []
for idx_row in range(len(unirep_values)):
    unirep_coupled = [rep_ for rep_ in unirep_values[idx_row]]
    unirep_col_value.append(np.array(unirep_coupled))

dict_result = {'sequences': sequences}
dict_result["unirep"] = unirep_col_value
pd.DataFrame(dict_result).to_pickle('results/df_unirep.pkl')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use tf.random.categorical instead.


pandas bar: 100%|██████████| 8716/8716 [1:34:02<00:00,  1.54it/s]


# ESM-2

In [ ]:
# sequence length < 2100 AAs, calculating with A800 with 80GB GPU memory
# # ESM-2
# import warnings
# from esm2.run_calculate_esm2 import run_calculate
# import pandas as pd
# from tqdm import tqdm
#
# # 过滤掉所有DeprecationWarning
# warnings.filterwarnings('ignore')
#
# df_all = pd.read_json('../ori_data/df_multi_tasks.json')
# input_sequences = df_all['sequence'].drop_duplicates()
# print(f'number of sequences : {len(input_sequences)}')
#
# protein_names, sequences = [], []
# for count, value in enumerate(tqdm(input_sequences)):
#     line=value.strip('/n')
#     if count % 2 == 0:
#         protein_names.append(line)
#     else:
#         if len(line) < 2100:
#             sequences.append(line)
#         else:
#             protein_names = protein_names[:-1]
#
# print(f"length of protein sequences: {len(sequences)}")
# df_result = pd.DataFrame({'sequences': sequences})
#
# seq_data = [(protein_name_, protein_sequence_) for protein_name_, protein_sequence_ in zip(protein_names, sequences)]
#
# esm2_1280 = run_calculate(seq_data, batch_size=1)
# df_result['esm2'] = esm2_1280
# df_result.to_pickle('/results/df_esm2.pkl')

In [2]:
import pandas as pd
df_esm2 = pd.read_pickle('./results/df_esm2.pkl')
print(len(df_esm2))
df_esm2.head()

16778


,sequences,esm2
0,MSIIKSYAAKEAGSELELYEYDAGELRPEDVEVQVDYCGICHSDLS...,"[-0.050770897, -0.036682352, -0.018163353, 0.0..."
1,MDIDRLFSVKGMNAVVLGASSGIGKAIAEMFSEMGGKVVLSDIDEE...,"[0.05781516, -0.055177405, 0.020610005, -0.031..."
2,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,"[0.04404524, -0.054761235, -0.0029805414, 0.03..."
3,MKAAVVREKNDGFVDLIDDWKPRELGFGDALVDVEYCGLCHTDLHC...,"[-0.016896145, -0.05097463, -0.039859887, 0.03..."
4,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,"[0.024043683, -0.13023601, -0.09008518, 0.0502..."


# ESM-C using python 3.10

In [1]:
from esm.models.esmc import ESMC
from esm.sdk.api import (
    ESMCInferenceClient,
    ESMProtein,
    LogitsConfig,
    LogitsOutput,
)
import pandas as pd
import warnings
from tqdm import tqdm
# 过滤掉所有DeprecationWarning
warnings.filterwarnings('ignore')


def main(client=ESMCInferenceClient, sequence=''):
    # ================================================================
    # Example usage: one single protein
    # ================================================================
    protein = ESMProtein(sequence=sequence)

    # Use logits endpoint. Using bf16 for inference optimization
    protein_tensor = client.encode(protein)
    output = client.logits(
        protein_tensor,
        LogitsConfig(sequence=True, return_embeddings=True, return_hidden_states=True),
    )
    assert isinstance(
        output, LogitsOutput
    ), f"LogitsOutput was expected but got {output}"
    assert output.logits is not None and output.logits.sequence is not None
    assert output.embeddings is not None
    assert output.hidden_states is not None


    embeddings = output.embeddings
    return embeddings


model = ESMC.from_pretrained("esmc_600m")

df_all = pd.read_json('../dataset/df_multi_tasks.json')
input_sequences = df_all['sequence'].drop_duplicates()
print(f'number of sequences : {len(input_sequences)}')

esmc_embeddings = []
for sequence in tqdm(input_sequences):
    embeddings = main(client=model, sequence=sequence).numpy().mean(axis=1)[0, :]
    esmc_embeddings.append(embeddings)

dict_result = {'sequences': input_sequences, 'esmc': esmc_embeddings}
pd.DataFrame(dict_result).to_json('results/df_esmc.json')

C:\Users\ching\anaconda3\envs\esmc\lib\site-packages\esm\tokenization\function_tokenizer.py:10: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  import scipy.sparse as sp


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

number of sequences : 16813


100%|██████████| 16813/16813 [6:11:16<00:00,  1.32s/it]   


## ProtT5- XL-Uniref50

In [1]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re
import pandas as pd
import numpy as np
import os, sys
current_dir = os.path.dirname(sys.argv[0])
from tqdm import tqdm, trange

def run_calculate(sequences, tokenizer, model, batch_size=2):
    embeddings = []
    error_sequences = []
    lengths = [len(seq) for seq in sequences]

    # replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
    sequence_preprocess = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequences]

    # batch
    for start_idx in trange(0, len(sequence_preprocess), batch_size):
        batch_sequences = sequence_preprocess[start_idx: start_idx + batch_size]
        length_of_sequences = lengths[start_idx: start_idx + batch_size]

        # tokenize sequences and pad up to the longest sequence in the batch
        ids = tokenizer(batch_sequences, add_special_tokens=True, padding="longest")

        input_ids = torch.tensor(ids['input_ids']).to(device)
        attention_mask = torch.tensor(ids['attention_mask']).to(device)

        try:
            # generate embeddings
            with torch.no_grad():
                embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)

            # extract residue embeddings for the sequences in the batch and remove padded & special tokens
            for idx_in_batch in range(len(batch_sequences)):
                embedding = embedding_repr.last_hidden_state[idx_in_batch, :length_of_sequences[idx_in_batch]].mean(dim=0)
                embeddings.append(embedding.cpu().numpy())
        except Exception as e:
            print(f"Error processing batch starting at index {start_idx}: {e}")
            error_sequences.extend(batch_sequences)

    df_result = pd.DataFrame({'sequences': sequences, 'prott5': embeddings})
    return df_result

# config
df_all = pd.read_json('../dataset/df_multi_tasks.json')
input_sequences = df_all['sequence'].drop_duplicates()
print(f'number of sequences : {len(input_sequences)}')

pre_models_path = './prott5/pre_models'

# model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device: ', device)
# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(pre_models_path, do_lower_case=False)
# Load the model
model = T5EncoderModel.from_pretrained(pre_models_path).to(device)

# calculate
df_result = run_calculate(input_sequences, tokenizer, model, batch_size=2)
df_result.to_json('./results/df_protT5.json')
df_result.head()

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


number of sequences : 16813
device:  cuda:0


100%|██████████| 8407/8407 [15:15<00:00,  9.18it/s]  


,sequences,prott5
0,MSIIKSYAAKEAGSELELYEYDAGELRPEDVEVQVDYCGICHSDLS...,"[0.016611475, 0.09350236, 0.009273678, 0.02968..."
2,MDIDRLFSVKGMNAVVLGASSGIGKAIAEMFSEMGGKVVLSDIDEE...,"[0.06827993, -0.029304748, 0.03184763, -0.0951..."
4,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,"[0.036032353, 0.019176442, 0.020855611, -0.072..."
17,MKAAVVREKNDGFVDLIDDWKPRELGFGDALVDVEYCGLCHTDLHC...,"[-0.0021131546, 0.005914332, -0.01885727, -5.2..."
18,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,"[0.032064147, 0.08067483, 0.024847964, -0.0083..."


# Prost-T5

In [1]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import pandas as pd
import re
from tqdm import trange

def run_calculate(sequences, tokenizer, model, batch_size=2):
    embeddings = []
    valid_sequences = []
    error_sequences = []
    lengths = [len(seq) for seq in sequences]

    sequences = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequences]
    sequences = [ "<AA2fold>" + " " + s if s.isupper() else "<fold2AA>" + " " + s  for s in sequences]

    # batch
    for start_idx in trange(0, len(sequences), batch_size):
        batch_sequences = sequences[start_idx: start_idx + batch_size]
        length_of_sequences = lengths[start_idx: start_idx + batch_size]

        # tokenize sequences and pad up to the longest sequence in the batch
        ids = tokenizer.batch_encode_plus(batch_sequences, add_special_tokens=True, padding="longest",return_tensors='pt').to(device)

        try:
            # generate embeddings
            with torch.no_grad():
                embedding_rpr = model(ids.input_ids, attention_mask=ids.attention_mask)

                # extract residue embeddings for the sequences in the batch and remove padded & special tokens
                for idx_in_batch in range(len(batch_sequences)):
                    embedding = embedding_rpr.last_hidden_state[idx_in_batch, :length_of_sequences[idx_in_batch]].mean(dim=0)
                    embeddings.append(embedding.cpu().numpy())

        except Exception as e:
            print(f"Error processing batch starting at index {start_idx}: {e}")
            error_sequences.extend(batch_sequences)

    df_result = pd.DataFrame({'sequences': input_sequences, 'prost5': embeddings})
    return df_result


# config
df_all = pd.read_json('../dataset/df_multi_tasks.json')
input_sequences = df_all['sequence'].drop_duplicates()
print(f'number of sequences : {len(input_sequences)}')

# model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('./prostt5', do_lower_case=False)
model = T5EncoderModel.from_pretrained("./prostt5").to(device)

# calculate
df_result = run_calculate(input_sequences, tokenizer, model, batch_size=1)
df_result.to_json('./results/df_prostT5.json')
df_result.head()

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


number of sequences : 16813
device:  cuda:0


100%|██████████| 16813/16813 [14:29<00:00, 19.34it/s] 


,sequences,prost5
0,MSIIKSYAAKEAGSELELYEYDAGELRPEDVEVQVDYCGICHSDLS...,"[0.0017113135, -0.030387186, -0.0036463677, 0...."
2,MDIDRLFSVKGMNAVVLGASSGIGKAIAEMFSEMGGKVVLSDIDEE...,"[-0.0041703084, -0.047611132, -0.0069104224, -..."
4,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,"[0.014685851, -0.013136651, 0.0020611903, -0.0..."
17,MKAAVVREKNDGFVDLIDDWKPRELGFGDALVDVEYCGLCHTDLHC...,"[0.006702779, -0.02026054, 0.007960504, -0.008..."
18,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,"[0.030726543, -0.020222707, -0.009790693, 0.02..."
